In [13]:
import logging
from pprint import pprint
import boto3
from botocore.exceptions import ClientError
from rekognition_objects import RekognitionFace
from rekognition_image_detection import RekognitionImage
from os import environ

logger = logging.getLogger(__name__)

In [2]:
class RekognitionCollection:
    """
    Encapsulates an Amazon Rekognition collection. This class is a thin wrapper
    around parts of the Boto3 Amazon Rekognition API.
    """
    def __init__(self, collection, rekognition_client):
        """
        Initializes a collection object.

        :param collection: Collection data in the format returned by a call to
                           create_collection.
        :param rekognition_client: A Boto3 Rekognition client.
        """
        self.collection_id = collection['CollectionId']
        self.collection_arn, self.face_count, self.created = self._unpack_collection(
            collection)
        self.rekognition_client = rekognition_client

    @staticmethod
    def _unpack_collection(collection):
        """
        Unpacks optional parts of a collection that can be returned by
        describe_collection.

        :param collection: The collection data.
        :return: A tuple of the data in the collection.
        """
        return (
            collection.get('CollectionArn'),
            collection.get('FaceCount', 0),
            collection.get('CreationTimestamp'))

    def to_dict(self):
        """
        Renders parts of the collection data to a dict.

        :return: The collection data as a dict.
        """
        rendering = {
            'collection_id': self.collection_id,
            'collection_arn': self.collection_arn,
            'face_count': self.face_count,
            'created': self.created
        }
        return rendering

    def describe_collection(self):
        """
        Gets data about the collection from the Amazon Rekognition service.

        :return: The collection rendered as a dict.
        """
        try:
            response = self.rekognition_client.describe_collection(
                CollectionId=self.collection_id)
            # Work around capitalization of Arn vs. ARN
            response['CollectionArn'] = response.get('CollectionARN')
            (self.collection_arn, self.face_count,
             self.created) = self._unpack_collection(response)
            logger.info("Got data for collection %s.", self.collection_id)
        except ClientError:
            logger.exception("Couldn't get data for collection %s.", self.collection_id)
            raise
        else:
            return self.to_dict()

    def delete_collection(self):
        """
        Deletes the collection.
        """
        try:
            self.rekognition_client.delete_collection(CollectionId=self.collection_id)
            logger.info("Deleted collection %s.", self.collection_id)
            self.collection_id = None
        except ClientError:
            logger.exception("Couldn't delete collection %s.", self.collection_id)
            raise

    def index_faces(self, image, max_faces):
        """
        Finds faces in the specified image, indexes them, and stores them in the
        collection.

        :param image: The image to index.
        :param max_faces: The maximum number of faces to index.
        :return: A tuple. The first element is a list of indexed faces.
                 The second element is a list of faces that couldn't be indexed.
        """
        try:
            response = self.rekognition_client.index_faces(
                CollectionId=self.collection_id, Image=image.image,
                ExternalImageId=image.image_name, MaxFaces=max_faces,
                DetectionAttributes=['ALL'])
            indexed_faces = [
                RekognitionFace({**face['Face'], **face['FaceDetail']})
                for face in response['FaceRecords']]
            unindexed_faces = [
                RekognitionFace(face['FaceDetail'])
                for face in response['UnindexedFaces']]
            logger.info(
                "Indexed %s faces in %s. Could not index %s faces.", len(indexed_faces),
                image.image_name, len(unindexed_faces))
        except ClientError:
            logger.exception("Couldn't index faces in image %s.", image.image_name)
            raise
        else:
            return indexed_faces, unindexed_faces

    def list_faces(self, max_results):
        """
        Lists the faces currently indexed in the collection.

        :param max_results: The maximum number of faces to return.
        :return: The list of faces in the collection.
        """
        try:
            response = self.rekognition_client.list_faces(
                CollectionId=self.collection_id, MaxResults=max_results)
            faces = [RekognitionFace(face) for face in response['Faces']]
            logger.info(
                "Found %s faces in collection %s.", len(faces), self.collection_id)
        except ClientError:
            logger.exception(
                "Couldn't list faces in collection %s.", self.collection_id)
            raise
        else:
            return faces

    def search_faces_by_image(self, image, threshold, max_faces):
        """
        Searches for faces in the collection that match the largest face in the
        reference image.

        :param image: The image that contains the reference face to search for.
        :param threshold: The match confidence must be greater than this value
                          for a face to be included in the results.
        :param max_faces: The maximum number of faces to return.
        :return: A tuple. The first element is the face found in the reference image.
                 The second element is the list of matching faces found in the
                 collection.
        """
        try:
            response = self.rekognition_client.search_faces_by_image(
                CollectionId=self.collection_id, Image=image.image,
                FaceMatchThreshold=threshold, MaxFaces=max_faces)
            image_face = RekognitionFace({
                'BoundingBox': response['SearchedFaceBoundingBox'],
                'Confidence': response['SearchedFaceConfidence']
            })
            collection_faces = [
                RekognitionFace(face['Face']) for face in response['FaceMatches']]
            logger.info("Found %s faces in the collection that match the largest "
                        "face in %s.", len(collection_faces), image.image_name)
        except ClientError:
            logger.exception(
                "Couldn't search for faces in %s that match %s.", self.collection_id,
                image.image_name)
            raise
        else:
            return image_face, collection_faces

    def search_faces(self, face_id, threshold, max_faces):
        """
        Searches for faces in the collection that match another face from the
        collection.

        :param face_id: The ID of the face in the collection to search for.
        :param threshold: The match confidence must be greater than this value
                          for a face to be included in the results.
        :param max_faces: The maximum number of faces to return.
        :return: The list of matching faces found in the collection. This list does
                 not contain the face specified by `face_id`.
        """
        try:
            response = self.rekognition_client.search_faces(
                CollectionId=self.collection_id, FaceId=face_id,
                FaceMatchThreshold=threshold, MaxFaces=max_faces)
            faces = [RekognitionFace(face['Face']) for face in response['FaceMatches']]
            logger.info(
                "Found %s faces in %s that match %s.", len(faces), self.collection_id,
                face_id)
        except ClientError:
            logger.exception(
                "Couldn't search for faces in %s that match %s.", self.collection_id,
                face_id)
            raise
        else:
            return faces

    def delete_faces(self, face_ids):
        """
        Deletes faces from the collection.

        :param face_ids: The list of IDs of faces to delete.
        :return: The list of IDs of faces that were deleted.
        """
        try:
            response = self.rekognition_client.delete_faces(
                CollectionId=self.collection_id, FaceIds=face_ids)
            deleted_ids = response['DeletedFaces']
            logger.info(
                "Deleted %s faces from %s.", len(deleted_ids), self.collection_id)
        except ClientError:
            logger.exception("Couldn't delete faces from %s.", self.collection_id)
            raise
        else:
            return deleted_ids

In [3]:
class RekognitionCollectionManager:
    """
    Encapsulates Amazon Rekognition collection management functions.
    This class is a thin wrapper around parts of the Boto3 Amazon Rekognition API.
    """
    def __init__(self, rekognition_client):
        """
        Initializes the collection manager object.

        :param rekognition_client: A Boto3 Rekognition client.
        """
        self.rekognition_client = rekognition_client

    def create_collection(self, collection_id):
        """
        Creates an empty collection.

        :param collection_id: Text that identifies the collection.
        :return: The newly created collection.
        """
        try:
            response = self.rekognition_client.create_collection(
                CollectionId=collection_id)
            response['CollectionId'] = collection_id
            collection = RekognitionCollection(response, self.rekognition_client)
            logger.info("Created collection %s.", collection_id)
        except ClientError:
            logger.exception("Couldn't create collection %s.", collection_id)
            raise
        else:
            return collection

    def list_collections(self, max_results):
        """
        Lists collections for the current account.

        :param max_results: The maximum number of collections to return.
        :return: The list of collections for the current account.
        """
        try:
            response = self.rekognition_client.list_collections(MaxResults=max_results)
            collections = [
                RekognitionCollection({'CollectionId': col_id}, self.rekognition_client)
                for col_id in response['CollectionIds']]
        except ClientError:
            logger.exception("Couldn't list collections.")
            raise
        else:
            return collections

In [4]:
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

In [5]:
rekognition_client = boto3.client('rekognition')

INFO: Found credentials in shared credentials file: ~/.aws/credentials


In [8]:
images = [
        RekognitionImage.from_file(
            'images/data01.jpg', rekognition_client,
            image_name='sitting'),
        RekognitionImage.from_file(
            'images/data02.jpg', rekognition_client,
            image_name='hopping'),
        RekognitionImage.from_file(
            'images/data03.jpg', rekognition_client,
            image_name='biking')]

In [17]:
collection_mgr = RekognitionCollectionManager(rekognition_client)
collection = collection_mgr.create_collection('doc-example-collection-demo')
print(f"Created collection {collection.collection_id}:")
pprint(collection.describe_collection())

INFO: Created collection doc-example-collection-demo.
INFO: Got data for collection doc-example-collection-demo.


Created collection doc-example-collection-demo:
{'collection_arn': 'arn:aws:rekognition:us-east-1:089715336747:collection/doc-example-collection-demo',
 'collection_id': 'doc-example-collection-demo',
 'created': datetime.datetime(2022, 9, 12, 20, 18, 27, 976000, tzinfo=tzlocal()),
 'face_count': 0}


In [18]:
print("Indexing faces from three images:")
for image in images:
    collection.index_faces(image, 10)

Indexing faces from three images:


INFO: Indexed 3 faces in sitting. Could not index 0 faces.
INFO: Indexed 2 faces in hopping. Could not index 1 faces.
INFO: Indexed 3 faces in biking. Could not index 0 faces.


In [19]:
print("Listing faces in collection:")
faces = collection.list_faces(10)
for face in faces:
    pprint(face.to_dict())

INFO: Found 8 faces in collection doc-example-collection-demo.


Listing faces in collection:
{'bounding_box': {'Height': 0.05230399966239929,
                  'Left': 0.5142009854316711,
                  'Top': 0.4133940041065216,
                  'Width': 0.033563900738954544},
 'face_id': '0ea3debe-7f27-4e86-a5a2-cec154afc364',
 'image_id': '6a4af9f0-38e9-3797-98fb-490688b87a96'}
{'bounding_box': {'Height': 0.06860210001468658,
                  'Left': 0.5122920274734497,
                  'Top': 0.2063249945640564,
                  'Width': 0.03965099900960922},
 'face_id': '17f4c797-f428-439f-b51c-6507611cc501',
 'image_id': 'e90468dc-9fa1-353c-9267-8b287b8deb6d'}
{'bounding_box': {'Height': 0.06226009875535965,
                  'Left': 0.5168030261993408,
                  'Top': 0.6083589792251587,
                  'Width': 0.03173350170254707},
 'face_id': '22092d66-998c-4225-82df-5ee03ea9dcdf',
 'image_id': '84d6d94a-da31-39dd-901d-74dd5eaa3201'}
{'bounding_box': {'Height': 0.05588740110397339,
                  'Left': 0.34624499082

In [20]:
print(f"Searching for faces in the collection that match the first face in the "
      f"list (Face ID: {faces[0].face_id}.")
found_faces = collection.search_faces(faces[0].face_id, 80, 10)
print(f"Found {len(found_faces)} matching faces.")
for face in found_faces:
    pprint(face.to_dict())

INFO: Found 2 faces in doc-example-collection-demo that match 0ea3debe-7f27-4e86-a5a2-cec154afc364.


Searching for faces in the collection that match the first face in the list (Face ID: 0ea3debe-7f27-4e86-a5a2-cec154afc364.
Found 2 matching faces.
{'bounding_box': {'Height': 0.06541279703378677,
                  'Left': 0.6546130180358887,
                  'Top': 0.3479740023612976,
                  'Width': 0.03941959887742996},
 'face_id': 'e7fa441d-661b-473d-a76f-3c2632b83ebc',
 'image_id': 'e90468dc-9fa1-353c-9267-8b287b8deb6d'}
{'bounding_box': {'Height': 0.06226009875535965,
                  'Left': 0.5168030261993408,
                  'Top': 0.6083589792251587,
                  'Width': 0.03173350170254707},
 'face_id': '22092d66-998c-4225-82df-5ee03ea9dcdf',
 'image_id': '84d6d94a-da31-39dd-901d-74dd5eaa3201'}


In [21]:
print(f"Searching for faces in the collection that match the largest face in "
      f"{images[0].image_name}.")
image_face, match_faces = collection.search_faces_by_image(images[0], 80, 10)
print(f"The largest face in {images[0].image_name} is:")
pprint(image_face.to_dict())
print(f"Found {len(match_faces)} matching faces.")
for face in match_faces:
    pprint(face.to_dict())

Searching for faces in the collection that match the largest face in sitting.


INFO: Found 3 faces in the collection that match the largest face in sitting.


The largest face in sitting is:
{'bounding_box': {'Height': 0.052303995937108994,
                  'Left': 0.5142008066177368,
                  'Top': 0.41339370608329773,
                  'Width': 0.03356394171714783}}
Found 3 matching faces.
{'bounding_box': {'Height': 0.05230399966239929,
                  'Left': 0.5142009854316711,
                  'Top': 0.4133940041065216,
                  'Width': 0.033563900738954544},
 'face_id': '0ea3debe-7f27-4e86-a5a2-cec154afc364',
 'image_id': '6a4af9f0-38e9-3797-98fb-490688b87a96'}
{'bounding_box': {'Height': 0.06541279703378677,
                  'Left': 0.6546130180358887,
                  'Top': 0.3479740023612976,
                  'Width': 0.03941959887742996},
 'face_id': 'e7fa441d-661b-473d-a76f-3c2632b83ebc',
 'image_id': 'e90468dc-9fa1-353c-9267-8b287b8deb6d'}
{'bounding_box': {'Height': 0.06226009875535965,
                  'Left': 0.5168030261993408,
                  'Top': 0.6083589792251587,
                  'Width

In [22]:
collection.delete_collection()

INFO: Deleted collection doc-example-collection-demo.


In [23]:
def detect_labels_local_file(photo):
    client=boto3.client('rekognition')
   
    with open(photo, 'rb') as image:
        response = client.detect_labels(Image={'Bytes': image.read()})
        
    print('Detected labels in ' + photo)    
    for label in response['Labels']:
        print (label['Name'] + ' : ' + str(label['Confidence']))

    return len(response['Labels'])

In [28]:
photo = 'images/data04.jpg'
label_count = detect_labels_local_file(photo)
print("Labels detected: " + str(label_count))

Detected labels in images/data04.jpg
Flag : 99.80938720703125
Symbol : 99.80938720703125
Person : 97.58856964111328
Human : 97.58856964111328
Shorts : 91.00053405761719
Clothing : 91.00053405761719
Apparel : 91.00053405761719
Field : 79.60782623291016
American Flag : 77.86731719970703
Labels detected: 9
